# 🛍️ Base de Datos Vectorial de Productos de Moda con Milvus

Este notebook demuestra cómo cargar y usar una base de datos vectorial completa con **30,000 productos de moda** de Flipkart.

## 🎯 Características principales:
- **Búsqueda semántica**: Encuentra productos usando lenguaje natural
- **Filtrado avanzado**: Por precio, marca, categoría, rating, etc.
- **Embeddings vectoriales**: Representación semántica de productos
- **Escalabilidad**: Maneja grandes volúmenes de datos eficientemente

## 📊 Dataset:
- **Fuente**: Flipkart Fashion Products Dataset 
- **Total productos**: 30,000 pero solo se cargaron 100 por la limitantes de Milvus
- **Campos**: Título, descripción, marca, categoría, precios, ratings, etc.
- **Modelo de embeddings**: sentence-transformers (768 dimensiones)


## 🔧 Paso 1: Instalación y Configuración


In [1]:
%pip install -U pymilvus pymilvus-model tqdm pandas 

Note: you may need to restart the kernel to use updated packages.


# Importar Librerias

In [2]:
# Importar librerías
from pymilvus import MilvusClient, model
import pandas as pd
import json
from typing import List, Dict

print("✅ Dependencias instaladas correctamente")

/Users/abrahamsolorzanopenaloza/miniconda3/envs/mioti_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


✅ Dependencias instaladas correctamente


In [7]:
# Instalar dependencias necesarias
%pip install -U pymilvus pymilvus-model tqdm pandas 

# Importar librerías
from pymilvus import MilvusClient, model
import pandas as pd
import json
from typing import List, Dict

print("✅ Dependencias instaladas correctamente")


Note: you may need to restart the kernel to use updated packages.


/Users/abrahamsolorzanopenaloza/miniconda3/envs/mioti_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


✅ Dependencias instaladas correctamente


# Conexión y Búsquedas Semánticas

Una vez que los datos están cargados, puedes realizar búsquedas semánticas avanzadas.


In [4]:
# Configurar conexión a Milvus
client = MilvusClient(
    uri="https://in03-fc51ec1067f53e7.serverless.gcp-us-west1.cloud.zilliz.com",
    token="e1a5f13be33e07ef22a239ffbc7ddffb08ada6250f4c7edfa0221784b304ac41627ed2db02a0e8be6e26c4e2f6b0db0d2ba2da09"
)

# Inicializar modelo de embeddings
embedding_fn = model.DefaultEmbeddingFunction()
collection_name = "flipkart_fashion_sample"

# Verificar que la colección existe
if client.has_collection(collection_name):
    print(f"✅ Conectado a la colección '{collection_name}'")
    
    # Obtener estadísticas básicas
    sample = client.query(
        collection_name=collection_name,
        filter="id >= 0",
        output_fields=["id"],
        limit=1
    )
    
    print(f"📊 Base de datos lista para búsquedas")
else:
    print("❌ La colección no existe. Ejecuta primero el paso de carga de datos.")


✅ Conectado a la colección 'flipkart_fashion_sample'
📊 Base de datos lista para búsquedas


In [5]:
# Función helper para búsquedas semánticas
def search_products(query: str, limit: int = 5, filters: str = None):
    """
    Realiza búsqueda semántica de productos
    """
    print(f"🔍 Buscando: '{query}'")
    print("-" * 60)
    
    # Generar vector de la consulta
    query_vector = embedding_fn.encode_queries([query])
    
    # Realizar búsqueda
    results = client.search(
        collection_name=collection_name,
        data=query_vector,
        limit=limit,
        filter=filters,
        output_fields=[
            "title", "description", "brand", "category", 
            "sub_category", "selling_price", "actual_price", 
            "average_rating", "discount", "out_of_stock"
        ]
    )
    
    if results and len(results[0]) > 0:
        print(f"✅ Encontrados {len(results[0])} resultados:\n")
        
        for i, result in enumerate(results[0], 1):
            product = result['entity']
            similarity = 1 - result['distance']  # Convertir distancia a similitud
            
            print(f"#{i} {product.get('title', 'Sin título')}")
            print(f"    🏷️  Marca: {product.get('brand', 'N/A')}")
            print(f"    📁 Categoría: {product.get('category', 'N/A')} > {product.get('sub_category', 'N/A')}")
            print(f"    💰 Precio: ${product.get('selling_price', 0):.0f} (antes: ${product.get('actual_price', 0):.0f})")
            print(f"    🎯 Descuento: {product.get('discount', 'N/A')}")
            print(f"    ⭐ Rating: {product.get('average_rating', 0)}")
            print(f"    📦 Stock: {'❌ Agotado' if product.get('out_of_stock', False) else '✅ Disponible'}")
            print(f"    🎯 Similitud: {similarity:.3f}")
            
            # Mostrar descripción truncada
            desc = product.get('description', '')
            if desc:
                desc_short = desc[:100] + "..." if len(desc) > 100 else desc
                print(f"    📝 Descripción: {desc_short}")
            
            print()
    else:
        print("❌ No se encontraron resultados")

print("✅ Función de búsqueda configurada")


✅ Función de búsqueda configurada


## 🎯 Ejemplos de Búsqueda Semántica

Ahora puedes realizar búsquedas usando lenguaje natural. La base de datos entenderá el contexto y encontrará productos similares.


In [7]:
# Ejemplo 1: Búsqueda básica
search_products("camisa azul para hombre", limit=10)


🔍 Buscando: 'camisa azul para hombre'
------------------------------------------------------------
✅ Encontrados 10 resultados:

#1 Striped Collared Neck Casual Men Brown, Black Sweater
    🏷️  Marca: Man
    📁 Categoría: Clothing and Accessories > Winter Wear
    💰 Precio: $399 (antes: $2199)
    🎯 Descuento: 81  off
    ⭐ Rating: 3.8
    📦 Stock: ✅ Disponible
    🎯 Similitud: 0.694
    📝 Descripción: MANRA present you stylish wollen ponchos, shawls,Cardgan, Pullovers which can be wear in any occasio...

#2 Solid V Neck Casual Men Purple Sweater
    🏷️  Marca: Man
    📁 Categoría: Clothing and Accessories > Winter Wear
    💰 Precio: $499 (antes: $2199)
    🎯 Descuento: 77  off
    ⭐ Rating: 3.0
    📦 Stock: ✅ Disponible
    🎯 Similitud: 0.711
    📝 Descripción: MANRA present you stylish wollen ponchos, shawls,Cardgan, Pullovers which can be wear in any occasio...

#3 Striped Collared Neck Casual Men Black Sweater
    🏷️  Marca: Man
    📁 Categoría: Clothing and Accessories > Winter We

In [8]:
# Ejemplo 2: Búsqueda con filtros de precio
search_products("zapatos deportivos", limit=3, filters="selling_price < 1000")


🔍 Buscando: 'zapatos deportivos'
------------------------------------------------------------
✅ Encontrados 3 resultados:

#1 Solid Men Black Track Pants
    🏷️  Marca: SMART ATTRACTI
    📁 Categoría: Clothing and Accessories > Bottomwear
    💰 Precio: $288 (antes: $1199)
    🎯 Descuento: 75  off
    ⭐ Rating: 4.7
    📦 Stock: ✅ Disponible
    🎯 Similitud: 0.796
    📝 Descripción: SMART ATTRACTION AIR Jogger Trackpants for men - for a sporty move gear up, get fit and enjoy a heal...

#2 Aztec Men Round or Crew Green, Black T-Shirt
    🏷️  Marca: Fairdea
    📁 Categoría: Clothing and Accessories > Topwear
    💰 Precio: $284 (antes: $1299)
    🎯 Descuento: 78  off
    ⭐ Rating: 2.6
    📦 Stock: ✅ Disponible
    🎯 Similitud: 0.803

#3 Sporty Men Round Neck Black T-Shirt
    🏷️  Marca: REEB
    📁 Categoría: Clothing and Accessories > Topwear
    💰 Precio: $919 (antes: $1899)
    🎯 Descuento: 51  off
    ⭐ Rating: 3.6
    📦 Stock: ✅ Disponible
    🎯 Similitud: 0.807
    📝 Descripción: TS Ar

In [9]:
# Ejemplo 3: Búsquedas más específicas
search_products("vestido elegante para fiesta", limit=2)
print("\n" + "="*60 + "\n")
search_products("ropa cómoda de algodón", limit=2)


🔍 Buscando: 'vestido elegante para fiesta'
------------------------------------------------------------
✅ Encontrados 2 resultados:

#1 nu-Lite Satin Tie   Cufflink (Red)
    🏷️  Marca: 
    📁 Categoría: Clothing and Accessories > Clothing Accessories
    💰 Precio: $599 (antes: $3299)
    🎯 Descuento: 81  off
    ⭐ Rating: 2.7
    📦 Stock: ✅ Disponible
    🎯 Similitud: 0.753
    📝 Descripción: Wearing an elegant man tie is an effective way to make you look charming and energetic. nu-Lite incl...

#2 nu-Lite Satin Tie   Cufflink (Red)
    🏷️  Marca: 
    📁 Categoría: Clothing and Accessories > Clothing Accessories
    💰 Precio: $499 (antes: $3299)
    🎯 Descuento: 84  off
    ⭐ Rating: 2.7
    📦 Stock: ✅ Disponible
    🎯 Similitud: 0.754
    📝 Descripción: Wearing an elegant man tie is an effective way to make you look charming and energetic. nu-Lite incl...



🔍 Buscando: 'ropa cómoda de algodón'
------------------------------------------------------------
✅ Encontrados 2 resultados:

